### Importing Necessary Libraries

In [ ]:
# --- File and path handling ---
import os  # For working with directories and file paths
import json  # For reading and writing JSON files (e.g., AOI GeoJSON)
import re  # For using regular expressions (e.g., extracting dates from filenames)

# --- Raster and spatial data processing ---
import rasterio  # For reading and handling raster (satellite) images
from rasterstats import (
    zonal_stats,
)  # For computing statistics of raster data over polygons

# --- Date and numerical operations ---
from datetime import datetime  # For parsing and handling dates
import numpy as np  # For numerical computations and array operations

# --- Tabular and geospatial data handling ---
import pandas as pd  # For storing and manipulating tabular data (DataFrames)
import geopandas as gpd  # For handling vector spatial data (polygons, CRS transformations)

### Initializing Necessary Paths

In [2]:
BASE_DIR = "../data"
AOI_PATH = os.path.join(BASE_DIR, "aoi.geojson")
OUTPUT_CSV = os.path.join(BASE_DIR, "all-landsat-data.csv")

In [3]:
# Loading our aoi.geojson file
with open(AOI_PATH) as f:
    aoi_geojson = json.load(f)

### Defining Some Important Functions 

In [4]:
def parse_metadata(path):
    """
    Reads a metadata file and extracts key-value pairs into a dictionary.
    """
    meta = {}
    with open(path) as f:
        for line in f:
            if "=" in line:
                k, v = line.strip().split("=")
                meta[k.strip()] = v.strip().strip('"')
    return meta

In [5]:
def read_band(path):
    """Read a single raster band and return array and profile."""
    with rasterio.open(path) as src:
        arr = src.read(1).astype(float)
        profile = src.profile
    return arr, profile


In [6]:
def scene_date_from_filename(fname):
    # example LC08_L2SP_141041_20230404_20230412_02_T1_SR_B4.TIF
    m = re.search(r"_(\d{8})_", fname)
    if m:
        return datetime.strptime(m.group(1), "%Y%m%d").date()
    return None

### NDVI (Normalized Difference Vegetation Index)

NDVI = $\frac{\text{NIR} - \text{Red}}{\text{NIR} + \text{Red}}$

Where:  
- **NIR** = Near-Infrared band reflectance  
- **Red** = Red band reflectance  

NDVI values range from **-1 to 1**, with higher values indicating healthier vegetation.

In [7]:
# Compute NDVI (Normalized Difference Vegetation Index) from NIR and Red bands
def compute_ndvi(nir, red):
    np.seterr(divide="ignore", invalid="ignore")
    ndvi = (nir - red) / (nir + red)
    ndvi = np.where(np.isfinite(ndvi), ndvi, np.nan)
    return ndvi

### NDWI (Normalized Difference Water Index)

NDWI = $\frac{\text{NIR} - \text{SWIR}}{\text{NIR} + \text{SWIR}}$

Where:  
- **NIR** = Near-Infrared band reflectance  
- **SWIR** = Short-Wave Infrared band reflectance  

NDWI values typically range from **-1 to 1**, with higher values indicating **more water content**.


In [8]:
# Compute NDWI (Normalized Difference Water Index) from NIR and SWIR bands
def compute_ndwi(nir, swir):
    np.seterr(divide="ignore", invalid="ignore")
    ndwi = (nir - swir) / (nir + swir)
    ndwi = np.where(np.isfinite(ndwi), ndwi, np.nan)
    return ndwi


### NDBI (Normalized Difference Built-up Index)


NDBI = $\frac{\text{SWIR} - \text{NIR}}{\text{SWIR} + \text{NIR}}$

Where:  
- **SWIR** = Short-Wave Infrared band reflectance  
- **NIR** = Near-Infrared band reflectance  
- NDBI ranges from **-1 to 1**, with higher values indicating **more built-up/urban areas**


In [9]:
# Compute NDBI (Normalized Difference Built-up Index) from SWIR and NIR band
def compute_ndbi(swir, nir):
    np.seterr(divide="ignore", invalid="ignore")
    ndbi = (swir - nir) / (swir + nir)
    ndbi = np.where(np.isfinite(ndbi), ndbi, np.nan)
    return ndbi

## Land Surface Temperature (LST) Conversion


LST (K) = $\text{B10} \times \text{mult} + \text{add}$

LST (°C) = $\text{LST (K)} - 273.15$


Where:  
- **B10** = thermal band digital number  
- **mult, add** = scaling factors from metadata  
- **LST** = Land Surface Temperature in Kelvin or Celsius

In [10]:
# Convert satellite thermal band (B10) values to Land Surface Temperature (LST) in Celsius
def compute_lst(b10, mult=0.00341802, add=149.0):
    """
    Convert ST_B10 scaled values to LST in Celsius.
    """
    lst_kelvin = b10 * mult + add
    lst_c = lst_kelvin - 273.15
    return lst_c


In [11]:
# Compute mean, standard deviation, and count of an index within a given AOI using zonal statistics
def mean_index_in_aoi(index_arr, profile, aoi_geojson):
    """
    Compute mean, std, count over AOI using zonal_stats.
    """
    stats = zonal_stats(
        aoi_geojson,
        index_arr,
        affine=profile["transform"],
        stats=["mean", "std", "count"],
        nodata=np.nan,
    )
    return stats[0]

In [12]:
# Process a single satellite scene folder to compute NDVI, NDWI, NDBI, LST and their statistics over the AOI
def process_scene_folder(scene_folder, aoi_geojson):
    # find band files
    b4 = next(p for p in os.listdir(scene_folder) if "_SR_B4" in p)
    b5 = next(p for p in os.listdir(scene_folder) if "_SR_B5" in p)
    b6 = next(p for p in os.listdir(scene_folder) if "_SR_B6" in p)
    b10 = next(p for p in os.listdir(scene_folder) if "_ST_B10" in p)

    # read bands
    b4_arr, prof = read_band(os.path.join(scene_folder, b4))
    b5_arr, _ = read_band(os.path.join(scene_folder, b5))
    b6_arr, _ = read_band(os.path.join(scene_folder, b6))
    b10_arr, _ = read_band(os.path.join(scene_folder, b10))

    # compute indices
    ndvi = compute_ndvi(b5_arr, b4_arr)
    ndwi = compute_ndwi(b5_arr, b6_arr)
    ndbi = compute_ndbi(b6_arr, b5_arr)
    lst_c = compute_lst(b10_arr)

    scene_date = scene_date_from_filename(b4)

    # reproject AOI to raster CRS
    gdf = gpd.GeoDataFrame.from_features(aoi_geojson["features"], crs="EPSG:4326")
    gdf_utm = gdf.to_crs(prof["crs"])
    aoi_geojson_utm = gdf_utm.__geo_interface__

    # compute stats
    ndvi_stats = mean_index_in_aoi(ndvi, prof, aoi_geojson_utm)
    ndwi_stats = mean_index_in_aoi(ndwi, prof, aoi_geojson_utm)
    ndbi_stats = mean_index_in_aoi(ndbi, prof, aoi_geojson_utm)
    lst_stats = mean_index_in_aoi(lst_c, prof, aoi_geojson_utm)

    return {
        "date": scene_date,
        "NDVI_mean": ndvi_stats["mean"],
        "NDVI_std": ndvi_stats["std"],
        "NDWI_mean": ndwi_stats["mean"],
        "NDWI_std": ndwi_stats["std"],
        "NDBI_mean": ndbi_stats["mean"],
        "NDBI_std": ndbi_stats["std"],
        "LST_mean_C": lst_stats["mean"],
        "LST_std_C": lst_stats["std"],
        "count": ndvi_stats["count"],  # single count column
    }


In [13]:
# Build timeseries for all scenes 
def build_timeseries(data_root, aoi_geojson):
    rows = []
    for year in os.listdir(data_root):
        ypath = os.path.join(data_root, year)
        if not os.path.isdir(ypath):
            continue
        for month in os.listdir(ypath):
            mpath = os.path.join(ypath, month)
            if not os.path.isdir(mpath):
                continue
            try:
                r = process_scene_folder(mpath, aoi_geojson)
                r["year"] = year
                r["scene"] = month
                rows.append(r)
            except StopIteration:
                continue
        
    df = pd.DataFrame(rows).dropna().sort_values("date").reset_index(drop=True)
    return df


In [14]:
# Build time series data
df_timeseries = build_timeseries(BASE_DIR, aoi_geojson)
df_timeseries

,date,NDVI_mean,NDVI_std,NDWI_mean,NDWI_std,NDBI_mean,NDBI_std,LST_mean_C,LST_std_C,count,year,scene
0,2022-01-27,0.201626,0.077391,0.055569,0.065216,-0.055569,0.065216,12.678688,4.144818,2649889,2022,January
1,2022-02-12,0.183799,0.099400,0.056023,0.066603,-0.056023,0.066603,13.168473,4.372898,2649889,2022,Feburary
2,2022-03-08,0.214993,0.070530,0.039716,0.068622,-0.039716,0.068622,25.446893,3.893588,2649889,2022,March
3,2022-04-17,0.243295,0.076340,0.059367,0.078458,-0.059367,0.078458,31.615345,4.400812,2649889,2022,April
4,2022-05-27,0.289823,0.087954,0.107981,0.066525,-0.107981,0.066525,27.281522,4.171977,2649889,2022,May
5,2022-10-18,0.303173,0.086264,0.128518,0.063930,-0.128518,0.063930,22.788756,3.496320,2649889,2022,October
6,2022-11-11,0.256337,0.086271,0.094939,0.068455,-0.094939,0.068455,20.941806,3.707592,2649889,2022,November
7,2022-12-05,0.225362,0.090743,0.070151,0.069436,-0.070151,0.069436,17.101854,3.469435,2649889,2022,December
8,2023-01-14,0.197984,0.085816,0.045536,0.070446,-0.045536,0.070446,16.092884,3.819108,2649889,2023,January
9,2023-02-07,0.183328,0.071969,0.037204,0.066612,-0.037204,0.066612,17.521065,4.480862,2649889,2023,Feburary


In [15]:
# save the built time series data
df_timeseries.to_csv(OUTPUT_CSV, index=False)